In [1]:
import requests, json
import pandas as pd
pd.options.display.max_rows = 999

import time, os
import numpy as np

import random
import time

import requests
import json
from bs4 import BeautifulSoup
import PyDST

In [2]:


url='https://www.jobindex.dk/jobsoegning.json?q=&jobage=archive&mindate=20070822&maxdate=20090822&page=2'
response = requests.get(url)
html=response.json()['result_list_box_html']
soup = BeautifulSoup(html,'lxml')
table_node = soup.find('div', attrs={'class':'results component--default'}) # find table node with specific attribute
table_node
#data.soup.find('div', attrs={'id':'results'})

<div class="results component--default">
<div data-beacon-tid="r3234069"><div class="jix_robotjob jix_job_archived">
<strong>Efterskolelærer søges hurtigst muligt!</strong>
<br/>
    Efterskolelærer søges hurtigst muligt!
    <br/>
<b>Staby Efterskole</b>, <b>Holstebro</b>
<br/>
<cite>Jobzonen, 22. august 2009</cite>
<div style="clear:both;"></div>
<span class="jix_toolbar jix_appetizer_toolbar" id="jix_toolbar_r3234069">
<ul class="toolbar-line toolbar-line-primary">
<li class="toolbar-myjob toolbar-myjob-notapplied">
<a data-action="add-dialog" href="/bruger/dine-job/r3234069/gem" rel="nofollow">
<span class="visible-xs-inline">Gem</span>
<span class="hidden-xs">Gem job</span>
</a> </li>
<li class="toolbar-follow hidden-xs jix_companyid_27331 jix_companyindex_follow">
<a class="follow_on" href="/bruger/dine-virksomheder?companyid=27331" rel="nofollow">Følg</a>
<a class="follow_off" href="/bruger/dine-virksomheder/27331/slet" rel="nofollow">Følger</a>
</li> <li class="toolbar-share"><

In [3]:
type(firm_city)

NameError: name 'firm_city' is not defined

In [ ]:
jobtitle_egne=table_node.find_all('a')
print(jobtitle_egne)

In [ ]:
jobtitle=table_node.find_all('strong')
citation=table_node.find_all('cite')
firm_city=table_node.find_all('b')
print(firm_city)

#kode til importerede annoncer for firma og by.

firm=[]
city=[]
cc=0

for i in firm_city:
    if cc %2==0:
        firm.append(i.text)
    else :
        city.append(i.text)
    cc += 1
print(firm)
print(city)

# BNP data

In [3]:
conn = PyDST.connection(language = 'en')

In [5]:
res = conn.get_tables('06')
res.info

Getting all tables under topic code(s) {topics}


,ID,Description,Variables,Unit,Last updated,First observation,Latest observation,Active
0,PRIS111,Forbrugerprisindeks,"[varegruppe, enhed, tid]",-,2018-08-10T08:00:00,2001M01,2018M07,True
1,PRIS112,Forbrugerprisindeks,"[hovedtal, tid]",pct.,2018-01-10T08:00:00,1980,2017,True
2,PRIS8,"Forbrugerprisindeks, årsgennemsnit","[type, tid]",indeks,2018-01-10T08:00:00,1900,2017,True
3,PRIS9,"Forbrugerprisindeks, gennemsnitlig årlig infla...","[type, tid]",pct.,2018-01-10T08:00:00,1900,2017,True
4,PRIS113,"Forbrugerprisindeks, hovedtal","[type, tid]",indeks,2018-08-10T08:00:00,1980M01,2018M07,True
5,PRIS200,Husstandsopdelte forbrugerprisindeks,"[varegruppe, husstandsgrupper, tid]",indeks,2018-06-29T08:00:00,2006M01,2017M12,True
6,PRIS114,Nettoprisindeks,"[varegruppe, enhed, tid]",indeks,2018-08-10T08:00:00,2001M01,2018M07,True
7,PRIS115,Nettoprisindeks,"[hovedtal, tid]",indeks,2018-01-10T08:00:00,1980,2017,True
8,PRIS116,"Nettoprisindeks, hovedtal","[type, tid]",indeks,2018-08-10T08:00:00,1980M01,2018M07,True
9,PPP,Købekraftpariteter og internationale mængde- o...,"[varegruppe, land, enhed, tid]",-,2017-12-20T08:00:00,2000,2016,True


In [6]:
conn.get_metadata('NKN1').unit

Getting metadata for table NKN1


'-'

In [7]:
resp = conn.get_data(
                table_id = 'NKN1',
                variables = ['transakt', 'prisenhed', 'sæson', 'tid'])
#data = resp.df


            No values selected! Setting values to all ('*')
            
Getting table NKN1, variables are ['transakt', 'prisenhed', 'sæson', 'tid']
        values are {'transakt': ['*'], 'prisenhed': ['*'], 'sæson': ['*'], 'tid': ['*']}


In [8]:
data = resp.df
data

,TRANSAKT,PRISENHED,SÆSON,TID,INDHOLD
0,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1990Q1,210.2
1,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1990Q2,218.1
2,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1990Q3,209.5
3,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1990Q4,217.8
4,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1991Q1,220.2
5,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1991Q2,226.1
6,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1991Q3,219.6
7,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1991Q4,224.6
8,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1992Q1,228.1
9,B.1*g Gross domestic product,"Current prices, (bill. DKK.)",Non-seasonally adjusted,1992Q2,231.1


In [9]:
data['PRISENHED'].unique()

array(['Current prices, (bill. DKK.)',
       'Period-to-period real growth in per cent',
       'Pr. capita. Current prices, (1000 DKK.)',
       '2010-prices, chained values, (bill. DKK.)',
       'Contribution to GDP growth, (percentage point)',
       'Pr. capita, 2010-prices, chained values, (1000 DKK.)'],
      dtype=object)

In [10]:
BNP=data[(data['PRISENHED']=='2010-prices, chained values, (bill. DKK.)') & 
     (data['TRANSAKT']=='B.1*g Gross domestic product') & 
     (data['SÆSON']=='Seasonally adjusted') ]

In [11]:
BNP.columns=['TRANSAKT', 'PRICE UNIT', 'SEASON', 'TIME', 'GDP, BILLIONS OF DKK']

In [12]:
BNP

,TRANSAKT,PRICE UNIT,SEASON,TIME,"GDP, BILLIONS OF DKK"
791,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q1,..
792,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q2,..
793,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q3,..
794,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q4,..
795,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1991Q1,326.6
796,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1991Q2,325.4
797,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1991Q3,329.5
798,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1991Q4,325.2
799,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1992Q1,331.1
800,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1992Q2,329.1


In [13]:
BNP['YEAR']=BNP['TIME'].apply(lambda x: int(x.split('Q')[0]))


/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
BNP['QUARTER']=BNP['TIME'].apply(lambda x: int(x.split('Q')[1]))

/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
BNP

,TRANSAKT,PRICE UNIT,SEASON,TIME,"GDP, BILLIONS OF DKK",YEAR,QUARTER
791,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q1,..,1990,1
792,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q2,..,1990,2
793,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q3,..,1990,3
794,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1990Q4,..,1990,4
795,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1991Q1,326.6,1991,1
796,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1991Q2,325.4,1991,2
797,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1991Q3,329.5,1991,3
798,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1991Q4,325.2,1991,4
799,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1992Q1,331.1,1992,1
800,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,1992Q2,329.1,1992,2


In [16]:
BNP[BNP['YEAR']>= 2008].to_csv('test.csv')

In [17]:
BNP[BNP['YEAR'].isin([2008,2009])]

,TRANSAKT,PRICE UNIT,SEASON,TIME,"GDP, BILLIONS OF DKK",YEAR,QUARTER
863,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2008Q1,474.8,2008,1
864,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2008Q2,470.4,2008,2
865,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2008Q3,467.6,2008,3
866,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2008Q4,456.6,2008,4
867,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2009Q1,450.1,2009,1
868,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2009Q2,441.4,2009,2
869,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2009Q3,442.9,2009,3
870,B.1*g Gross domestic product,"2010-prices, chained values, (bill. DKK.)",Seasonally adjusted,2009Q4,443.3,2009,4


# Beskæftigelsesdata

In [18]:
resp_1 = conn.get_data(
                table_id = 'AKU103',
                variables = ['beskstatus', 'tid'])




            No values selected! Setting values to all ('*')
            
Getting table AKU103, variables are ['beskstatus', 'tid']
        values are {'beskstatus': ['*'], 'tid': ['*']}


In [19]:
data_1 = resp_1.df
data_1

,BESKSTATUS,TID,INDHOLD
0,Employed,1996Q1,2597
1,Employed,1996Q2,2612
2,Employed,1996Q3,2617
3,Employed,1996Q4,2623
4,Employed,1997Q1,2635
5,Employed,1997Q2,2658
6,Employed,1997Q3,2668
7,Employed,1997Q4,2674
8,Employed,1998Q1,2667
9,Employed,1998Q2,2660


In [20]:
conn.get_metadata('AKU103').variables

Getting metadata for table AKU103


['beskstatus', 'tid']

In [21]:
data_1.columns=['EMPLOY_STATUS', 'TIME', 'EMPLOYED, (1000 per.)']

In [22]:
EMPLOY=data_1[(data_1['EMPLOY_STATUS']=='Employed') ]

In [23]:
EMPLOY['YEAR']=EMPLOY['TIME'].apply(lambda x: int(x.split('Q')[0]))

/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
EMPLOY['QUARTER']=EMPLOY['TIME'].apply(lambda x: int(x.split('Q')[1]))

/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
EMPLOY

,EMPLOY_STATUS,TIME,"EMPLOYED, (1000 per.)",YEAR,QUARTER
0,Employed,1996Q1,2597,1996,1
1,Employed,1996Q2,2612,1996,2
2,Employed,1996Q3,2617,1996,3
3,Employed,1996Q4,2623,1996,4
4,Employed,1997Q1,2635,1997,1
5,Employed,1997Q2,2658,1997,2
6,Employed,1997Q3,2668,1997,3
7,Employed,1997Q4,2674,1997,4
8,Employed,1998Q1,2667,1998,1
9,Employed,1998Q2,2660,1998,2


In [26]:
EMPLOY[EMPLOY['YEAR']>= 2008].to_csv('test_employ.csv')

In [27]:
EMPLOY[EMPLOY['YEAR'].isin([2008,2009])]

,EMPLOY_STATUS,TIME,"EMPLOYED, (1000 per.)",YEAR,QUARTER
48,Employed,2008Q1,2771,2008,1
49,Employed,2008Q2,2812,2008,2
50,Employed,2008Q3,2820,2008,3
51,Employed,2008Q4,2818,2008,4
52,Employed,2009Q1,2774,2009,1
53,Employed,2009Q2,2733,2009,2
54,Employed,2009Q3,2727,2009,3
55,Employed,2009Q4,2673,2009,4


In [28]:
EMPLOY.drop(['EMPLOY_STATUS', 'TIME'], axis=1,inplace=True)

/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# BAROMETER for industrien

In [29]:
conn.get_metadata('BARO8').variables

Getting metadata for table BARO8


['indikator', 'sæson', 'tid']

In [30]:
resp_2 = conn.get_data(
                table_id = 'BARO8',
                variables = ['indikator', 'sæson', 'tid'])



            No values selected! Setting values to all ('*')
            
Getting table BARO8, variables are ['indikator', 'sæson', 'tid']
        values are {'indikator': ['*'], 'sæson': ['*'], 'tid': ['*']}


In [31]:
data_2 = resp_2.df
data_2

,INDIKATOR,SÆSON,TID,INDHOLD
0,"Confidence indicator, total",Non-seasonally adjusted,2005M01,1
1,"Confidence indicator, total",Non-seasonally adjusted,2005M02,4
2,"Confidence indicator, total",Non-seasonally adjusted,2005M03,3
3,"Confidence indicator, total",Non-seasonally adjusted,2005M04,-6
4,"Confidence indicator, total",Non-seasonally adjusted,2005M05,-2
5,"Confidence indicator, total",Non-seasonally adjusted,2005M06,-4
6,"Confidence indicator, total",Non-seasonally adjusted,2005M07,-5
7,"Confidence indicator, total",Non-seasonally adjusted,2005M08,7
8,"Confidence indicator, total",Non-seasonally adjusted,2005M09,3
9,"Confidence indicator, total",Non-seasonally adjusted,2005M10,2


In [32]:
BAR_IND=data_2[(data_2['INDIKATOR']=='Confidence indicator, total') &
               (data_2['SÆSON']=='Seasonally adjusted')]

In [33]:
BAR_IND['YEAR']=BAR_IND['TID'].apply(lambda x: int(x.split('M')[0]))
BAR_IND['QUARTER']=BAR_IND['TID'].apply(lambda x: int(x.split('M')[1]))

/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
BAR_IND.drop(['INDIKATOR', 'SÆSON'], axis=1,inplace=True)


/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [35]:
BAR_IND

,TID,INDHOLD,YEAR,QUARTER
163,2005M01,3,2005,1
164,2005M02,2,2005,2
165,2005M03,0,2005,3
166,2005M04,-6,2005,4
167,2005M05,-2,2005,5
168,2005M06,-3,2005,6
169,2005M07,-6,2005,7
170,2005M08,5,2005,8
171,2005M09,1,2005,9
172,2005M10,2,2005,10


In [36]:
BAR_IND.columns=['TIME', 'CONFIDENCE_IND_TOTAL_IND', 'YEAR', 'QUARTER']
BAR_IND.to_csv('test_bar_ind.csv')

# BAROMETER for Bygge og anlægsvirksomhed

In [37]:
conn.get_metadata('KBYG44').variables


Getting metadata for table KBYG44


['indikator', 'sæson', 'tid']

In [38]:
resp_3 = conn.get_data(
                table_id = 'KBYG44',
                variables = ['indikator', 'sæson', 'tid'])


            No values selected! Setting values to all ('*')
            
Getting table KBYG44, variables are ['indikator', 'sæson', 'tid']
        values are {'indikator': ['*'], 'sæson': ['*'], 'tid': ['*']}


In [39]:
data_3 = resp_3.df
data_3

,INDIKATOR,SÆSON,TID,INDHOLD
0,"Confidence indicator, total",Non-seasonally adjusted,1998M01,-14
1,"Confidence indicator, total",Non-seasonally adjusted,1998M02,-7
2,"Confidence indicator, total",Non-seasonally adjusted,1998M03,5
3,"Confidence indicator, total",Non-seasonally adjusted,1998M04,17
4,"Confidence indicator, total",Non-seasonally adjusted,1998M05,13
5,"Confidence indicator, total",Non-seasonally adjusted,1998M06,13
6,"Confidence indicator, total",Non-seasonally adjusted,1998M07,9
7,"Confidence indicator, total",Non-seasonally adjusted,1998M08,5
8,"Confidence indicator, total",Non-seasonally adjusted,1998M09,1
9,"Confidence indicator, total",Non-seasonally adjusted,1998M10,-8


In [40]:
BAR_BYG=data_3[(data_3['INDIKATOR']=='Confidence indicator, total') &
               (data_3['SÆSON']=='Seasonally adjusted')]

In [41]:
BAR_BYG['YEAR']=BAR_BYG['TID'].apply(lambda x: int(x.split('M')[0]))
BAR_BYG['QUARTER']=BAR_BYG['TID'].apply(lambda x: int(x.split('M')[1]))

/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
BAR_BYG

,INDIKATOR,SÆSON,TID,INDHOLD,YEAR,QUARTER
247,"Confidence indicator, total",Seasonally adjusted,1998M01,-3,1998,1
248,"Confidence indicator, total",Seasonally adjusted,1998M02,-1,1998,2
249,"Confidence indicator, total",Seasonally adjusted,1998M03,5,1998,3
250,"Confidence indicator, total",Seasonally adjusted,1998M04,7,1998,4
251,"Confidence indicator, total",Seasonally adjusted,1998M05,6,1998,5
252,"Confidence indicator, total",Seasonally adjusted,1998M06,4,1998,6
253,"Confidence indicator, total",Seasonally adjusted,1998M07,1,1998,7
254,"Confidence indicator, total",Seasonally adjusted,1998M08,-2,1998,8
255,"Confidence indicator, total",Seasonally adjusted,1998M09,-4,1998,9
256,"Confidence indicator, total",Seasonally adjusted,1998M10,-5,1998,10


In [43]:
BAR_BYG.drop(['INDIKATOR', 'SÆSON'], axis=1,inplace=True)

/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [44]:
BAR_BYG.columns=['TIME', 'CONFIDENCE_IND_TOTAL_BYG', 'YEAR', 'QUARTER']
BAR_BYG

,TIME,CONFIDENCE_IND_TOTAL_BYG,YEAR,QUARTER
247,1998M01,-3,1998,1
248,1998M02,-1,1998,2
249,1998M03,5,1998,3
250,1998M04,7,1998,4
251,1998M05,6,1998,5
252,1998M06,4,1998,6
253,1998M07,1,1998,7
254,1998M08,-2,1998,8
255,1998M09,-4,1998,9
256,1998M10,-5,1998,10


In [45]:
BAR_BYG.to_csv('test_bar_byg.csv')

# BAROMETER for ... service og detail går kun tilbage til 2011

In [4]:
conn.get_metadata('KONCYK1').variables

Getting metadata for table KONCYK1


['erhverv', 'mængde4', 'tid']

In [7]:
resp_4 = conn.get_data(
                table_id = 'KONCYK1',
                variables = ['erhverv', 'mængde4', 'tid'])


            No values selected! Setting values to all ('*')
            
Getting table KONCYK1, variables are ['erhverv', 'mængde4', 'tid']
        values are {'erhverv': ['*'], 'mængde4': ['*'], 'tid': ['*']}


In [8]:
data_4 = resp_4.df
data_4

,ERHVERV,MÆNGDE4,TID,INDHOLD
0,Manufacturing industries,Indicator level,1998M02,1.24
1,Manufacturing industries,Indicator level,1998M03,1.05
2,Manufacturing industries,Indicator level,1998M04,0.86
3,Manufacturing industries,Indicator level,1998M05,0.65
4,Manufacturing industries,Indicator level,1998M06,0.44
5,Manufacturing industries,Indicator level,1998M07,0.21
6,Manufacturing industries,Indicator level,1998M08,-0.03
7,Manufacturing industries,Indicator level,1998M09,-0.28
8,Manufacturing industries,Indicator level,1998M10,-0.54
9,Manufacturing industries,Indicator level,1998M11,-0.79


In [10]:
KONJUN=data_4[(data_4['MÆNGDE4']=='Month over month') ]
KONJUN

,ERHVERV,MÆNGDE4,TID,INDHOLD
246,Manufacturing industries,Month over month,1998M02,-0.19
247,Manufacturing industries,Month over month,1998M03,-0.19
248,Manufacturing industries,Month over month,1998M04,-0.20
249,Manufacturing industries,Month over month,1998M05,-0.20
250,Manufacturing industries,Month over month,1998M06,-0.21
251,Manufacturing industries,Month over month,1998M07,-0.23
252,Manufacturing industries,Month over month,1998M08,-0.24
253,Manufacturing industries,Month over month,1998M09,-0.26
254,Manufacturing industries,Month over month,1998M10,-0.26
255,Manufacturing industries,Month over month,1998M11,-0.24


In [15]:
KONJUN['YEAR']=KONJUN['TID'].apply(lambda x: int(x.split('M')[0]))
KONJUN['MONTH']=KONJUN['TID'].apply(lambda x: int(x.split('M')[1]))
KONJUN

/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ERHVERV,MÆNGDE4,TID,INDHOLD,YEAR,QUARTER,MONTH
246,Manufacturing industries,Month over month,1998M02,-0.19,1998,2,2
247,Manufacturing industries,Month over month,1998M03,-0.19,1998,3,3
248,Manufacturing industries,Month over month,1998M04,-0.20,1998,4,4
249,Manufacturing industries,Month over month,1998M05,-0.20,1998,5,5
250,Manufacturing industries,Month over month,1998M06,-0.21,1998,6,6
251,Manufacturing industries,Month over month,1998M07,-0.23,1998,7,7
252,Manufacturing industries,Month over month,1998M08,-0.24,1998,8,8
253,Manufacturing industries,Month over month,1998M09,-0.26,1998,9,9
254,Manufacturing industries,Month over month,1998M10,-0.26,1998,10,10
255,Manufacturing industries,Month over month,1998M11,-0.24,1998,11,11


In [16]:
KONJUN.drop([ 'QUARTER'], axis=1,inplace=True)

/Users/mariannerasmussen/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [19]:
KONJUN.columns=['PROFESSION', 'MEASURING', 'TIME', 'CONTENT', 'YEAR', 'MONTH']
KONJUN
KONJUN[KONJUN['YEAR'].isin([2008,2009])]

,PROFESSION,MEASURING,TIME,CONTENT,YEAR,MONTH
365,Manufacturing industries,Month over month,2008M01,-0.10,2008,1
366,Manufacturing industries,Month over month,2008M02,-0.12,2008,2
367,Manufacturing industries,Month over month,2008M03,-0.14,2008,3
368,Manufacturing industries,Month over month,2008M04,-0.17,2008,4
369,Manufacturing industries,Month over month,2008M05,-0.21,2008,5
370,Manufacturing industries,Month over month,2008M06,-0.25,2008,6
371,Manufacturing industries,Month over month,2008M07,-0.30,2008,7
372,Manufacturing industries,Month over month,2008M08,-0.35,2008,8
373,Manufacturing industries,Month over month,2008M09,-0.38,2008,9
374,Manufacturing industries,Month over month,2008M10,-0.39,2008,10


In [20]:
KONJUN.to_csv('test_KONJUN.csv')